In [41]:
import sys
sys.path.append("../scripts")

import data
import experiments
import models
import utils


%load_ext autoreload
%autoreload 2

# load code profiling
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [18]:
dataset = "compas"
fold = 0
X_train, A_train, Y_train, X_val, A_val, Y_val, X_test, A_test, Y_test =  data.get_fold(dataset, fold, 10, 8, 0)

{'4_False': 0, '1_True': 1, '4_True': 2, '2_True': 3, '2_False': 4, '1_False': 5, '3_True': 6, '3_False': 7}


In [35]:
%%timeit
model = models.M2FGB(fair_weight = 0.4, n_estimators=5)
model.fit(X_train, Y_train, A_train)

467 ms ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
model = models.M2FGB(fair_weight = 0.4, n_estimators=5)
model.fit(X_train, Y_train, A_train)

22 ms ± 4.81 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
import lightgbm as lgb
import numpy as np

In [16]:
# fit

X = X_train.values
y = Y_train.values
sensitive_attribute = A_train.values

# sort X and y based on sensitive_attribute
idx = np.argsort(sensitive_attribute)
X = X[idx]
y = y[idx]
sensitive_attribute = sensitive_attribute[idx]


I = models.get_subgroup_indicator_test(sensitive_attribute)
subgroup = sensitive_attribute
fairness_constraint = "equalized_loss"
dual_learning = "gradient_norm"
fair_weight = 0.5
multiplier_learning_rate = 0.1


group_losses = []
mu_opt_list = [None]
dtrain = lgb.Dataset(X, label=y)


def custom_obj(predt, dtrain):
    y_true = dtrain.get_label()
    y_pred = 1 / (1 + np.exp(-predt))
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)  # avoid log(0)
    loss_group = models.logloss_group(y_pred, y_true, subgroup, fairness_constraint)
    group_losses.append(loss_group)

    if dual_learning == "optim":
        # dual problem solved analytically
        idx_biggest_loss = np.where(loss_group == np.max(loss_group))[0]
        # if is more than one, randomly choose one
        idx_biggest_loss = np.random.choice(idx_biggest_loss)
        mu_opt = np.zeros(loss_group.shape[0])
        mu_opt[idx_biggest_loss] = fair_weight

    elif dual_learning == "gradient":
        if mu_opt_list[0] is None:
            mu_opt = np.zeros(loss_group.shape[0])
        else:
            mu_opt = mu_opt_list[-1].copy()
        mu_opt += multiplier_learning_rate * fair_weight * loss_group

    elif dual_learning == "gradient_norm":
        if mu_opt_list[0] is None:
            mu_opt = np.ones(loss_group.shape[0])
        else:
            mu_opt = mu_opt_list[-1].copy()

        mu_opt += multiplier_learning_rate * loss_group
        mu_opt = models.projection_to_simplex(mu_opt, z=fair_weight)

    elif dual_learning == "gradient_norm2":
        if mu_opt_list[0] is None:
            mu_opt = np.ones(loss_group.shape[0])
        else:
            mu_opt = mu_opt_list[-1].copy()

        mu_opt += multiplier_learning_rate * loss_group
        mu_opt = mu_opt / np.sum(mu_opt) * fair_weight

    if mu_opt_list[0] is None:
        mu_opt_list[0] = mu_opt
    else:
        mu_opt_list.append(mu_opt)

    grad_fair = models.logloss_group_grad(y_pred, y_true, fairness_constraint)
    grad_fair = I * grad_fair.reshape(-1, 1) @ mu_opt

    hess_fair = models.logloss_group_hess(y_pred, y_true, fairness_constraint)
    hess_fair = I * hess_fair.reshape(-1, 1) @ mu_opt

    grad = models.logloss_grad(y_pred, y_true)
    hess = models.logloss_hessian(y_pred, y_true)

    # It is not necessary to multiply fairness gradient by fair_weight because it is already included on mu
    # grad = (1 - fair_weight) * grad + fair_weight * grad_fair
    # hess = (1 - fair_weight) * hess + fair_weight * hess_fair

    grad = (1 - fair_weight) * grad + grad_fair
    hess = (1 - fair_weight) * hess + hess_fair

    return grad, hess



params = {
    "objective": custom_obj,
    "learning_rate": 0.1,
    "max_depth": 6,
    "min_child_weight": 1,
    "reg_lambda": 1,
    "verbose": -1,
    "random_seed" : 0
}

def prof_function():
    model_ = lgb.train(
        params,
        dtrain,
        num_boost_round=100,
    )


In [17]:
%lprun -f custom_obj prof_function()

Timer unit: 1e-09 s

Total time: 0.366325 s
File: /tmp/ipykernel_9301/1815502925.py
Function: custom_obj at line 27

Line #      Hits         Time  Per Hit   % Time  Line Contents
    27                                           def custom_obj(predt, dtrain):
    28       100     178789.0   1787.9      0.0      y_true = dtrain.get_label()
    29       100   11331754.0 113317.5      3.1      y_pred = 1 / (1 + np.exp(-predt))
    30       100    3262954.0  32629.5      0.9      y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)  # avoid log(0)
    31       100   55101009.0 551010.1     15.0      loss_group = models.logloss_group(y_pred, y_true, subgroup, fairness_constraint)
    32       100     120497.0   1205.0      0.0      group_losses.append(loss_group)
    33                                           
    34       100      76874.0    768.7      0.0      if dual_learning == "optim":
    35                                                   # dual problem solved analytically
    36             